In [10]:
from googleplaces import GooglePlaces, types, lang
from googlemaps import Client as GoogleMaps
from gmplot import gmplot
import pickle
import time
from decimal import Decimal
import pandas as pd
# pip install gmplot
import os

near_By_Roads = []


In [9]:
def Read_API_KEY_From_File(file_Path):
    list_Key = []
    with open(file_Path) as file_text:
        for key in file_text:
            list_Key.append(key.strip())
    return list_Key

api_key_File_Path = r'D:\Product\API_KEY\API_KEY.txt'

list_Key = Read_API_KEY_From_File(api_key_File_Path)

PLACE_API_KEY = list_Key[0]
ROADS_API_KEY = list_Key[1]

file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'


google_places = GooglePlaces(PLACE_API_KEY)
gmaps = GoogleMaps(ROADS_API_KEY)

In [12]:
import pickle

list_Places_Delhi = []
file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'

with open(file_Path_To_Store_Data, 'rb') as f:
    list_Places_Delhi = pickle.load(f)
    
len(list_Places_Delhi)

550

In [10]:
# Place map
gmap = gmplot.GoogleMapPlotter(28.68, 77.13, 13)

In [11]:
all_Types_To_Search = [types.TYPE_PARKING, 
                       types.TYPE_TRAIN_STATION,
                       types.TYPE_TAXI_STAND, 
                       types.TYPE_GAS_STATION,
                       
                       types.TYPE_MOVIE_THEATER,
                       types.TYPE_AIRPORT, 
                       types.TYPE_SHOPPING_MALL, 
                       
#                        types.TYPE_TRANSIT_STATION,
#                        types.TYPE_RESTAURANT,
#                        types.TYPE_AMUSEMENT_PARK, 
#                        types.TYPE_AQUARIUM,
#                        types.TYPE_BEAUTY_SALON,
#                        types.TYPE_CAFE,
#                        types.TYPE_BOWLING_ALLEY,
                       
#                        types.TYPE_GROCERY_OR_SUPERMARKET,
#                        types.TYPE_GYM, 
#                        types.TYPE_LIBRARY,
#                        types.TYPE_HOSPITAL,
                       
#                        types.TYPE_PARK,
#                        types.TYPE_BAR,
                       
                       types.TYPE_GAS_STATION                      
                      ]

# Delhi Grid

In [5]:
# For whole Delhi 
lat_Lower_Range = 28.412593
lat_Upper_Range = 28.881338
long_Lower_Range = 76.838069
long_Upper_Range = 77.348458

# https://www.mapdevelopers.com/geocode_bounding_box.php

# For only New Delhi
# lat_Lower_Range = 28.404100
# lat_Upper_Range = 28.650480
# long_Lower_Range = 77.073010
# long_Upper_Range = 77.344960


kilo_Meter_To_From_Grid = 5

increment_Value = (1/110) * kilo_Meter_To_From_Grid


top_attraction_lats = []
top_attraction_lons = []

lat = lat_Lower_Range
lng = long_Lower_Range

count_Total_Lat_Lng_Created = 0

while lat <= lat_Upper_Range:
    lng = long_Lower_Range
    while lng <= long_Upper_Range:
        top_attraction_lats.append(lat)
        top_attraction_lons.append(lng)
        lng = lng + increment_Value
        count_Total_Lat_Lng_Created = count_Total_Lat_Lng_Created + 1
    
    lat = lat + increment_Value
   
print ("count_Total_Lat_Lng_Created ",count_Total_Lat_Lng_Created)

gmap = gmplot.GoogleMapPlotter(28.68, 77.13, 13)
gmap.scatter(top_attraction_lats, top_attraction_lons, color = '#ff0000 ', size = (kilo_Meter_To_From_Grid * 100), marker = False)

gmap.draw("my_map.html")

count_Total_Lat_Lng_Created  132


In [17]:
gmap.draw("my_map.html")

In [18]:
file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'
list_Places_Delhi = []
with open(file_Path_To_Store_Data, 'rb') as f:
    list_Places_Delhi = pickle.load(f)

# Delhi Places

In [13]:
top_attraction_lats = []
top_attraction_lons = []

place_Type_Count = []

for types in all_Types_To_Search:
    count = 0
    for place in list_Places_Delhi:
        if (place.types[0] == types):
            count = count + 1   
    place_Type_Count.append((types, count))
    
count = 0
for place in list_Places_Delhi:
    if (place.types[0] in all_Types_To_Search):
        count = count + 1
        top_attraction_lats.append(float(place.geo_location['lat']))
        top_attraction_lons.append(float(place.geo_location['lng']))
    
print ("Total Places ",(count))

print(place_Type_Count)

gmap = gmplot.GoogleMapPlotter(28.68, 77.13, 13)

gmap.scatter(top_attraction_lats, top_attraction_lons, color = '#ff0000 ', size = (2 * 100), marker = False)

gmap.draw("my_map.html")

Total Places  167
[('parking', 38), ('train_station', 15), ('taxi_stand', 24), ('gas_station', 31), ('movie_theater', 20), ('airport', 27), ('shopping_mall', 12), ('gas_station', 31)]


# Nearby Roads


In [44]:
def Get_Nearest_Road_Details(location_To_Search):
    result = gmaps.nearest_roads(location_To_Search)
    
    if (len(result) > 0):
        placeId = str(result[0]['placeId'])
        place_Object = google_places.get_place(placeId)
        return (place_Object)
    else:
        return False

In [29]:
top_attraction_lats = []
top_attraction_lons = []

lat_Near_By_Roads = []
lng_Near_By_Roads = []
    
count = 0

for place in list_Places_Delhi:
    if (place.types[0] in all_Types_To_Search):
        road_place = Get_Nearest_Road_Details(place.geo_location)
        
        if road_place != False:
            count = count + 1
            lat = float(place.geo_location['lat'])
            lng = float(place.geo_location['lng'])
            top_attraction_lats.append(lat)
            top_attraction_lons.append(lng)
            
            lat = float(road_place.geo_location['lat'])
            lng = float(road_place.geo_location['lng'])
            
            lat_Near_By_Roads.append(lat)
            lng_Near_By_Roads.append(lng)
            
            print (count)
            
       
print ("Total Places ",(count))

gmap = gmplot.GoogleMapPlotter(28.68, 77.13, 13)

gmap.scatter(top_attraction_lats, top_attraction_lons, color = '#ff0000', size = (2 * 100), marker = False)

gmap.scatter(lat_Near_By_Roads, lng_Near_By_Roads, color = '#0000FF', size = (2 * 100), marker = False)

gmap.draw("my_map.html")

In [48]:
gmap = gmplot.GoogleMapPlotter(28.68, 77.13, 13)

gmap.scatter(top_attraction_lats, top_attraction_lons, color = '#ff0000', size = 50, marker = False)

gmap.scatter(lat_Near_By_Roads, lng_Near_By_Roads, color = '#0000FF', size = (50), marker = False)

gmap.draw("my_map.html")

# Roads

In [30]:
source_Folder_Path = r'C:\Users\Sachin13390\Desktop\Python_Practice_Files\Nasscom_Data\Data\Vod For 9 Ward 95 Locations'

In [31]:
all_Files = []

for (dirpath, dirnames, filenames) in os.walk(source_Folder_Path):
    
    if (dirpath != folder_Path):
        for (dirpath, dirnames, filenames) in walk(dirpath):
            for file in filenames:
                if file.startswith('VDO'):
#                     print (os.path.join(dirpath, file))
                    all_Files.append(os.path.join(dirpath, file))

In [32]:
len(all_Files)

74

In [37]:
excel_file_Path = all_Files[0]

In [44]:
excel_file_Path

'C:\\Users\\Sachin13390\\Desktop\\Python_Practice_Files\\Nasscom_Data\\Data\\Vod For 9 Ward 95 Locations\\Road 1\\VDO Baner Pashan Link Road 5.xlsx'

In [69]:
df = pd.read_excel(excel_file_Path, sheet_name = 'UP')

df.head(100)

,Classified Traffic Volume Count Survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Location:,Baner Pashan Link Road,NaN,NaN,Date:,2017-12-28 00:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direction:,Towards Baner Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Time,Vehicle Category,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Fast Moving Passenger Vehicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Goods Vechiles,...,NaN,LCVS,NaN,NaN,Goods Auto,Tractor + Trailar,Army/Govt Vehicles/Ambulance,Cycle,Other,Animal Hand Drawn
5,NaN,Car/JeepVan(Private),Car/JeepVan(Taxi),3 Wheelers,Two Wheelers,S.T PMPML Buses,Private Bu,Institutional Buses,Mini Bus,2 Axle Trucks,...,Mutli Axle Truck,Type 1 (4 Tyrees),Type 2 (6 Tyrees),Type 3 (Tata Ace),NaN,NaN,NaN,NaN,NaN,NaN
6,08:00 - 08:15,27,9,4,61,NaN,NaN,NaN,3,NaN,...,NaN,NaN,NaN,2,NaN,NaN,NaN,5,NaN,1
7,08:15 - 08:30,33,5,6,60,1,NaN,1,2,NaN,...,NaN,1,1,NaN,1,NaN,NaN,4,NaN,NaN
8,08:30 - 08:45,37,7,4,80,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
9,08:45 - 09:00,40,12,4,148,1,NaN,NaN,1,NaN,...,NaN,2,1,NaN,NaN,NaN,NaN,3,NaN,NaN


In [70]:
df.iloc[5]['Unnamed: 1']

'Car/JeepVan(Private)'

In [93]:
start_Col_Value = 'Car/JeepVan(Private)'
total_Rows_To_Iterate = 12 # For 8-11 peak hours, for 4 session per hour
total_Cols_To_Iterate = 4

temp_cols = ['Car/JeepVan(Private)', 'Car/JeepVan(Taxi)','3 Wheelers','Two Wheelers']

new_df = pd.DataFrame( columns = temp_cols)

for excel_file_Path in all_Files:
    
    df = pd.read_excel(excel_file_Path, sheet_name = 'UP')

    for row_Index in range (len(df)):

        col_prefix = 'Unnamed: ' 
        start_Counter = 1

        col_name = col_prefix + str(start_Counter)

        if df.iloc[row_Index][col_name] == start_Col_Value:
            
            new_Row = []
            keep_Running = True

            for col_incrementer in range(start_Counter, (start_Counter + total_Cols_To_Iterate)):
                col_name = col_prefix + str(col_incrementer)

                if keep_Running :
                    value_count = 0
                    for current_Row_index in range ((row_Index + 1), (total_Rows_To_Iterate + row_Index + 1)):
                        if (type(df.iloc[current_Row_index][col_name]) is int) and keep_Running:
                            value_count = (df.iloc[current_Row_index][col_name]) + value_count
                        else:
                            keep_Running = False
                            break

                new_Row.append(value_count)

            if keep_Running :
                new_df.loc[len(new_df)] = new_Row     

            col_name = col_prefix + str(start_Counter)


In [94]:
# new_df.sort_values(by=['Car/JeepVan(Private)'] , ascending = False, inplace= True)
# new_df.reset_index(drop=True, inplace=True)

In [95]:
new_df.to_csv('Vehicle_Types_Data.csv', index = False)